# Setting enviroment

In [1]:
%load_ext kedro.ipython

[01/02/25 11:38:23] INFO     Using                                                                  ]8;id=838632;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/framework/project/__init__.py\__init__.py]8;;\:]8;id=736491;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/framework/project/__init__.py#270\270]8;;\
                             '/home/bruno/Documents/Programming/Programming_projects/olist_project/                
                             .venv/lib/python3.11/site-packages/kedro/framework/project/rich_loggin                
                             g.yml' as logging configuration.                                                      

                    INFO     Registered line magic '%reload_kedro'                                   ]8;id=387190;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=291333;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#61\61]8;;\

                    INFO     Registered line magic '%load_node'                                      ]8;id=422003;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=480414;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#63\63]8;;\

                    INFO     Resolved project path as:                                              ]8;id=278285;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=198633;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#178\178]8;;\
                             /home/bruno/Documents/Programming/Programming_projects/olist_project.                 
                             To set a different path, run '%reload_kedro <project_root>'                           

[01/02/25 11:38:40] INFO     Kedro is sending anonymous usage data with the sole purpose of improving ]8;id=826987;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py\plugin.py]8;;\:]8;id=954534;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro_telemetry/plugin.py#233\233]8;;\
                             the product. No personal data or IP addresses are stored on our side. If              
                             you want to opt out, set the `KEDRO_DISABLE_TELEMETRY` or `DO_NOT_TRACK`              
                             environment variables, or create a `.telemetry` file in the current                   
                             working directory with the contents `consent: false`. Read more at                    
                             https://docs.kedro.org/en/stable/configuration/telemetry.html                         

                    INFO     Kedro project olist_project                                            ]8;id=729837;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=15565;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#144\144]8;;\

                    INFO     Defined global variable 'context', 'session', 'catalog' and            ]8;id=370493;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=950156;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#145\145]8;;\
                             'pipelines'                                                                           

[01/02/25 11:38:41] INFO     Registered line magic 'run_viz'                                        ]8;id=261373;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=556041;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/ipython/__init__.py#151\151]8;;\

In [2]:
import pandas as pd

# Load data

In [3]:
df_customers = catalog.load("raw_customers")
df_geo = (
    catalog.load("raw_geolocation")
    .assign(
        geolocation_city = lambda df: (
            df
            .groupby('geolocation_zip_code_prefix')
            .geolocation_city.transform('first')
            .str.normalize('NFKD')
            .str.encode('ascii', errors='ignore')
            .str.decode('utf-8')
            .str.strip()
        ),
        geolocation_state = lambda df: (
            df
            .groupby('geolocation_zip_code_prefix')
            .geolocation_state.transform('first')
        )
    )
    .filter(['geolocation_zip_code_prefix', 'geolocation_city','geolocation_state'])
    .drop_duplicates()
)
df_items = catalog.load("raw_order_items")
df_pay = catalog.load("raw_order_payments")
df_rev = catalog.load("raw_order_reviews")
df_orders = catalog.load("raw_orders")
df_prd = catalog.load("raw_products")
df_sel = catalog.load("raw_sellers")


                    INFO     Loading data from raw_customers (ParquetDataset)...                ]8;id=842697;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=364691;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from raw_geolocation (ParquetDataset)...              ]8;id=940486;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=945133;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

[12/25/24 12:44:00] INFO     Loading data from raw_order_items (ParquetDataset)...              ]8;id=731098;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=487122;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from raw_order_payments (ParquetDataset)...           ]8;id=775311;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=547430;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from raw_order_reviews (ParquetDataset)...            ]8;id=814165;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=332315;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from raw_orders (ParquetDataset)...                   ]8;id=893224;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=232796;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

[12/25/24 12:44:01] INFO     Loading data from raw_products (ParquetDataset)...                 ]8;id=885046;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=857165;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

                    INFO     Loading data from raw_sellers (ParquetDataset)...                  ]8;id=796194;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=678908;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [4]:
df_pay

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45
...,...,...,...,...,...
103881,0406037ad97740d563a178ecc7a2075c,1,boleto,1,363.31
103882,7b905861d7c825891d6347454ea7863f,1,credit_card,2,96.80
103883,32609bbb3dd69b3c066a6860554a77bf,1,credit_card,1,47.77
103884,b8b61059626efa996a60be9bb9320e10,1,credit_card,5,369.54


In [5]:
for col in df_sel.columns:
    print(f"{col}: ")

seller_id: 
seller_zip_code_prefix: 
seller_city: 
seller_state: 


# Pagamentos de pedidos

In [185]:
print(
    "pode haver mais de um tipo de pagamento por pedido. "
    "\nno geral, esses pedidos com mais de um tipo de pagamento é por que o cliente "
    "\ncoloca cupons no momento da compra"
)
display(df_pay.groupby('order_id').payment_type.nunique().sort_values())
print("por exemplo")
display(df_pay.query('order_id=="62b2d948f0d2de4fe48b034bbdedf94c"'))

pode haver mais de um tipo de pagamento por pedido. 
no geral, esses pedidos com mais de um tipo de pagamento é por que o cliente 
coloca cupons no momento da compra



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
aa3d4b2c0fb6da178364768f0d3008f2    1
aa3d230ecbebaa35cea8fe4c24739dbc    1
aa3bb8a5c7740d386e6318f44046ac14    1
aa3babada56030c2ad79976c60d51e79    1
                                   ..
6a868efa34dccad934e939877994dafe    2
2dcb16f827ccfc7bff5be2a6deb28bce    2
0c3e2dc4a3eeb282bd4df0b620dcff6d    2
257006b4652ed6214a889913fec6a5ca    2
62b2d948f0d2de4fe48b034bbdedf94c    2
Name: payment_type, Length: 99440, dtype: int64

por exemplo


,order_id,payment_sequential,payment_type,payment_installments,payment_value
17668,62b2d948f0d2de4fe48b034bbdedf94c,2,voucher,1,250.00
59535,62b2d948f0d2de4fe48b034bbdedf94c,1,credit_card,8,87.52


In [188]:

print(
    "mesmo tirando os cupons ainda pode haver mais de um tipo de pagamento por pedido"
)
display(df_pay.query('payment_type!="voucher"').groupby('order_id').payment_type.nunique().sort_values())
print("mas isso só acontece em um pedido:")
display(df_pay.query('order_id=="a079628ac8002126e75f86b0f87332e4"').sort_values('payment_sequential'))

mesmo tirando os cupons ainda pode haver mais de um tipo de pagamento por pedido



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
ab5628545011511727a47e013585705d    1
ab53b19e9f59776c6556ebf49e85a52c    1
ab537f9c58a21747d30d02a43d2763a7    1
ab532e8d49a6cd8848ec641843b39b93    1
                                   ..
557de3912df820904b9204e0d1879256    1
557d6978e67b46040feebb5bf50e62d2    1
557cc2675910c4a6c9b54ad276f25097    1
557c1116cc609730b617567dec5790b3    1
a079628ac8002126e75f86b0f87332e4    2
Name: payment_type, Length: 97819, dtype: int64

mas isso só acontece em um pedido:


,order_id,payment_sequential,payment_type,payment_installments,payment_value
36471,a079628ac8002126e75f86b0f87332e4,2,debit_card,1,50.00
10258,a079628ac8002126e75f86b0f87332e4,3,credit_card,3,102.82


## Preprocessamento: Trazendo id de vendedor para pagamentos e info de data

In [75]:
df_pay_pre = (
    df_pay
    .merge(
        df_orders[["order_id", "order_status", "order_purchase_timestamp"]]
        .drop_duplicates(),
        on="order_id",
        how='left',
        validate='m:1'
    )
    .query("order_status=='delivered'")
    .drop(columns="order_status")
    .merge(
        df_items[['seller_id','order_id']]
        .drop_duplicates(),
        on=['order_id'],
        how='left',
        validate='m:m',
        indicator=True
    )
)
assert all(df_pay_pre._merge.astype("string").fillna("nulo") == "both")
df_pay_pre = df_pay_pre.drop(columns=['_merge'])

# Review de pedidos

In [ ]:
print("pode haver mais de um review para cada pedido")
display(df_rev.order_id.value_counts())
print("por exemplo")
df_rev.query('order_id=="8e17072ec97ce29f0e1f111e598b0c85"')


order_id
c88b1d1b157a9999ce368f218a407141    3
8e17072ec97ce29f0e1f111e598b0c85    3
df56136b8031ecd28e200bb18e6ddb2e    3
03c939fd7fd3b38f8485a0f95798f1f6    3
5cb890a68b91b6158d69257e4e2bc359    2
                                   ..
5b4e9a12d219f34f5c2de9f8d620b19d    1
a6da096d974acc000962856d7386448a    1
75e0647c26de647eca3421e9cc66c9da    1
bad0467c52f23cdc71e9fa139d4a8afd    1
90531360ecb1eec2a1fbb265a0db0508    1
Name: count, Length: 98673, dtype: int64

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
44694,67c2557eb0bd72e3ece1e03477c9dff5,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Entregou o produto errado.,2018-04-07 00:00:00,2018-04-08 22:48:27
64510,2d6ac45f859465b5c185274a1c929637,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,Comprei 3 unidades do produto vieram 2 unidade...,2018-04-07 00:00:00,2018-04-07 21:13:05
92300,6e4c4086d9611ae4cc0cc65a262751fe,8e17072ec97ce29f0e1f111e598b0c85,1,NaN,"Embora tenha entregue dentro do prazo, não env...",2018-04-14 00:00:00,2018-04-16 11:37:31


In [9]:
df_rev.isna().mean()


review_id                  0.000000
order_id                   0.000000
review_score               0.000000
review_comment_title       0.883415
review_comment_message     0.587025
review_creation_date       0.000000
review_answer_timestamp    0.000000
dtype: float64

In [7]:
assert all(df_rev.review_creation_date <= df_rev.review_answer_timestamp)

In [12]:
print("A dada review_answer_timestamp é posterior a review_creation_date")
display((pd.to_datetime(df_rev.review_answer_timestamp) - pd.to_datetime(df_rev.review_creation_date)).dt.days.describe([.05,.15,.25,.5,.75,.85,.95,.98]))
print("Usarei a data review_answer_timestamp, pois é posterior e, segundo o dicionario,")
print("indica o momento real em que a resposta foi enviada")

A dada review_answer_timestamp é posterior a review_creation_date



count    99224.000000
mean         2.582248
std          9.890526
min          0.000000
5%           0.000000
15%          0.000000
25%          1.000000
50%          1.000000
75%          3.000000
85%          3.000000
95%          6.000000
98%         13.000000
max        518.000000
dtype: float64

Usarei a data review_answer_timestamp, pois é posterior e, segundo o dicionario,
indica o momento real em que a resposta foi enviada


## Preprocessamento: Tranzendo id de vendedor

In [ ]:
df_rev_pre = (
    df_rev
    .merge(
        df_orders[["order_id", "order_status"]]
        .drop_duplicates(),
        on="order_id",
        how='left',
        validate='m:1'
    )
    .query("order_status=='delivered'")
    .drop(columns="order_status")
    .merge(
        df_items[['seller_id','order_id']]
        .drop_duplicates(),
        on=['order_id'],
        how='left',
        validate='m:m',
        indicator=True
    )
)
assert all(df_rev_pre._merge.astype("string").fillna("nulo") == "both")
df_rev_pre = df_rev_pre.drop(columns=['_merge','order_id'])

In [108]:
df_rev_pre

,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,seller_id
0,7bc2406110b926393aa56f80a40eba40,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,6d803cb79cc31c41c4c789a75933b3c7
1,80e641a11e56f04c1ad469d5645fdfde,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,8e6d7754bc7e0f22c96d255ebda59eba
2,228ce5500dc1d8e020d8d1322874b6f0,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,a1043bafd471dff536d0c462352beb48
3,e64fb393e7b32834bb789ff8bb30750e,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,a1043bafd471dff536d0c462352beb48
4,f7c4243c7fe1938f181bec41a392bdeb,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,989becdce12ebc39863c2bceab6f3ca1
...,...,...,...,...,...,...,...
97697,574ed12dd733e5fa530cfd4bbf39d7c9,5,NaN,NaN,2018-07-07 00:00:00,2018-07-14 17:18:30,5343d0649eca2a983820bfe93fc4d17e
97698,f3897127253a9592a73be9bdfdf4ed7a,5,NaN,NaN,2017-12-09 00:00:00,2017-12-11 20:06:42,46dc3b2cc0980fb8ec44634e21d2718e
97699,b3de70c89b1510c4cd3d0649fd302472,5,NaN,"Excelente mochila, entrega super rápida. Super...",2018-03-22 00:00:00,2018-03-23 09:10:43,17ca9b9e9b9ef8fdb529001b49ebb50f
97700,1adeb9d84d72fe4e337617733eb85149,4,NaN,NaN,2018-07-01 00:00:00,2018-07-02 12:59:13,8d956fec2e4337affcb520f56fd8cbfd


# Carrinhos de compras

In [7]:
df_items.columns


Index(['order_id', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value'],
      dtype='object')

In [27]:
print("granularidade da base de items:")
display(df_items[["order_id","order_item_id"]].value_counts())

print("Um mesmo pedido pode ter o mesmo produto repetido várias vezes.")
print("Exemplo de order id com mais de order_items:")
display( df_items.query("order_id=='8272b63d03f5f79c56e9e4120aec44ef'"))


granularidade da base de items:



order_id                          order_item_id
00010242fe8c5a6d1ba2dd792cb16214  1                1
ab114f47094abcacaf5888844db683ff  1                1
ab14fdcfbe524636d65ee38360e22ce8  4                1
                                  3                1
                                  2                1
                                                  ..
553a1f1b74cf15fe6cb4a01a627ab41f  1                1
553a0f2feba6fd288aaea2986ca2f94a  1                1
5539bd029cf95d97ba8f51f6f323c839  1                1
553901a853048dcd33ec8de19f90c5d0  1                1
fffe41c64501cc87c801fd61db3f6244  1                1
Name: count, Length: 112650, dtype: int64

Um mesmo pedido pode ter o mesmo produto repetido várias vezes.
Exemplo de order id com mais de order_items:


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
57297,8272b63d03f5f79c56e9e4120aec44ef,1,270516a3f41dc035aa87d220228f844c,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57298,8272b63d03f5f79c56e9e4120aec44ef,2,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57299,8272b63d03f5f79c56e9e4120aec44ef,3,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57300,8272b63d03f5f79c56e9e4120aec44ef,4,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57301,8272b63d03f5f79c56e9e4120aec44ef,5,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57302,8272b63d03f5f79c56e9e4120aec44ef,6,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57303,8272b63d03f5f79c56e9e4120aec44ef,7,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57304,8272b63d03f5f79c56e9e4120aec44ef,8,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57305,8272b63d03f5f79c56e9e4120aec44ef,9,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89
57306,8272b63d03f5f79c56e9e4120aec44ef,10,05b515fdc76e888aada3c6d66c201dff,2709af9587499e95e803a6498a5a56e9,2017-07-21 18:25:23,1.2,7.89


In [28]:

print("O produto com mesmo id só pode ter um único vendedor?")
print(
    df_items.groupby("product_id").seller_id.nunique().max()==1
)
print("Exemplo de produto com mais de um vendedor")
display(
    df_items
    .query("product_id=='d285360f29ac7fd97640bf0baef03de0'")
)

O produto com mesmo id só pode ter um único vendedor?
False
Exemplo de produto com mais de um vendedor


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
1070,027b81ccdf72434e7346aafdf630ff31,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 02:45:11,154.91,16.18
1692,03e2511c3ad8af9188ef59218b1e8a2c,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 03:05:12,154.91,16.18
3035,06d1315b8f0db57ee04b47b21db6626c,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 18:25:17,155.97,13.75
4228,0987363c91de454d42d084b84a691f99,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-25 15:06:12,155.97,8.35
6070,0dcaff4e23cee08bff0fe78261d1baaa,1,d285360f29ac7fd97640bf0baef03de0,4869f7a5dfa277a7dca6462dcf3b52b2,2018-03-16 08:31:02,329.00,20.18
...,...,...,...,...,...,...,...
107507,f44cb69655f8e4d13e7aae7cdd3d3c25,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-24 22:35:20,155.97,8.35
110288,fa939fb695a0a458a472916809af767d,1,d285360f29ac7fd97640bf0baef03de0,4869f7a5dfa277a7dca6462dcf3b52b2,2018-03-21 20:48:41,298.00,19.97
110480,fb0a15c6b3645581d8535f4f873c7110,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-20 04:30:20,194.91,8.62
110812,fbc32bc4a800352a5c9d8a57828b4727,1,d285360f29ac7fd97640bf0baef03de0,2eb70248d66e0e3ef83659f71b244378,2018-07-24 00:46:07,199.90,24.12


In [180]:
print('pode haver mais de um vendedor em cada pedido')
display(
    df_items
    .groupby('order_id').seller_id.nunique()
    .sort_values()
)
print("por exemplo: ")
display(df_items.query('order_id=="1c11d0f4353b31ac3417fbfa5f0f2a8a" '))

print("Mas para cada produto em um pedido há apenas um único vendedor :")
display(
    df_items
    .groupby(['order_id','product_id']).seller_id.nunique()
    .sort_values()
)

pode haver mais de um vendedor em cada pedido



order_id
00010242fe8c5a6d1ba2dd792cb16214    1
aaaf314a8cf0d0da71e52c6cd4184cbd    1
aaaea350ff8a957595f3c631d6b63d1b    1
aaae80f5b6239bd9e1b22e9aa542c3e8    1
aaabf43feb9498d9de4588eb73231c25    1
                                   ..
1d23106803c48c391366ff224513fb7f    4
8c2b13adf3f377c8f2b06b04321b0925    4
91be51c856a90d7efe86cf9d082d6ae3    4
cf5c8d9f52807cb2d2f0a0ff54c478da    5
1c11d0f4353b31ac3417fbfa5f0f2a8a    5
Name: seller_id, Length: 98666, dtype: int64

por exemplo: 


,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
12378,1c11d0f4353b31ac3417fbfa5f0f2a8a,1,d64738c7b2e2d1778a51e835d8f02568,609e1a9a6c2539919b8205cf7c4e6ff0,2018-02-02 11:31:03,43.90,12.77
12379,1c11d0f4353b31ac3417fbfa5f0f2a8a,2,a3a10562c9d134b92f04b8cce298d037,822166ed1e47908f7cfb49946d03c726,2018-02-02 11:31:03,22.49,25.54
12380,1c11d0f4353b31ac3417fbfa5f0f2a8a,3,d80fce9f775927c7d602c2de926759f1,747c1cdcd1737dcfbd1547f6eb6f2cf3,2018-02-02 11:31:03,88.00,12.77
12381,1c11d0f4353b31ac3417fbfa5f0f2a8a,4,ddbf57e4559f03c003fc3708d4860c5e,1835b56ce799e6a4dc4eddc053f04066,2018-02-02 11:31:03,29.99,12.77
12382,1c11d0f4353b31ac3417fbfa5f0f2a8a,5,a3a10562c9d134b92f04b8cce298d037,822166ed1e47908f7cfb49946d03c726,2018-02-02 11:31:03,22.49,25.54
12383,1c11d0f4353b31ac3417fbfa5f0f2a8a,6,d3a386ad58b99f0ddcc216ed9efb03c3,609e1a9a6c2539919b8205cf7c4e6ff0,2018-02-02 11:31:03,43.90,12.77
12384,1c11d0f4353b31ac3417fbfa5f0f2a8a,7,61b6e5d2e3ee58d2b341b8ef1a652b26,c7fdb77fdbff3c41981bc52f787e959e,2018-02-02 11:31:03,56.00,12.77


Mas para cada produto em um pedido há apenas um único vendedor :



order_id                          product_id                      
00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61    1
ab5048879a15f9f713725c37c5eb43ef  adae758556f8338f07f3064736416064    1
ab500cb1d60a0e2ae7ce02ef4e8236d9  8473872cbf0ba205b447e8651bd6e03e    1
ab4fc7cc026a9ea1a616584f9cfe4b9a  bb6b6e173472747c5b94c0b72c4fe56c    1
ab4f963cf3f826eda5b1b000a7af4de8  1279fba7025d91dadde108559a8d0485    1
                                                                     ..
556689f3e30f4662d3ec219bdc556b93  915ff35b0555b98fe64bc60674da4c43    1
5564c14d026bf6c92241f070bb98fb5b  b84c4c7f1845f068f40e350865b803fa    1
5561adcb0fd46da4cad3048fa4e7fc00  4d071a22b9cc548b64a75075a4c63151    1
556bb5c8ecb64be5ee1ee039d4376432  f5d5fa2bc95883494c61ae05a351348a    1
fffe41c64501cc87c801fd61db3f6244  350688d9dc1e75ff97be326363655e01    1
Name: seller_id, Length: 102425, dtype: int64

## Preprocessamento: Trazendo info de produtos e data, e filtrando para pedidos entregues

In [118]:

df_items_pre = (
    df_items
    .merge(df_prd,
           on=['product_id'],
           validate='m:1',
           how='left',
           indicator=True)
)

assert all(df_items_pre._merge=="both")
# o dataset resultante deve ter o mesmo tamnho 
assert len(df_items_pre) == len(df_items)
df_items_pre = (
    df_items_pre
    .merge(
        df_orders[["order_id", "order_status", "order_purchase_timestamp"]]
        .drop_duplicates(),
        on="order_id",
        how='left',
        validate='m:1'
    )
    .query("order_status=='delivered'")
)
df_items_pre = df_items_pre.drop(columns=['order_status','order_id', '_merge','order_item_id'])

In [119]:
df_items_pre

,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,order_purchase_timestamp
0,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,cool_stuff,58.0,598.0,4.0,650.0,28.0,9.0,14.0,2017-09-13 08:59:02
1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,pet_shop,56.0,239.0,2.0,30000.0,50.0,30.0,40.0,2017-04-26 10:53:06
2,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,moveis_decoracao,59.0,695.0,2.0,3050.0,33.0,13.0,33.0,2018-01-14 14:33:31
3,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,perfumaria,42.0,480.0,1.0,200.0,16.0,10.0,15.0,2018-08-08 10:00:35
4,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,ferramentas_jardim,59.0,409.0,1.0,3750.0,35.0,40.0,30.0,2017-02-04 13:57:51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112645,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,utilidades_domesticas,43.0,1002.0,3.0,10150.0,89.0,15.0,40.0,2018-04-23 13:57:06
112646,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,informatica_acessorios,31.0,232.0,1.0,8950.0,45.0,26.0,38.0,2018-07-14 10:26:46
112647,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,esporte_lazer,43.0,869.0,1.0,967.0,21.0,24.0,19.0,2017-10-23 17:07:56
112648,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,informatica_acessorios,56.0,1306.0,1.0,100.0,20.0,20.0,20.0,2017-08-14 23:02:59


# Clientes

In [45]:
df_customers.columns


Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [202]:
print("só existem 1 customer_id por linha na base")
display(df_customers.customer_id.value_counts())
print("Mas podem ter vários customer_unique_id para cada linha")
display(df_customers.customer_unique_id.value_counts())
print("por exemplo:")
display(df_customers.query("customer_unique_id=='8d50f5eadf50201ccdcedfb9e2ac8455'"))

só existem 1 customer_id por linha na base



customer_id
06b8999e2fba1a1fbc88172c00ba8bc7    1
c023f30c1147aeb0358474f3b1dbc707    1
b5cbf43f42281920a175fc99650c91d6    1
19f4e983f8f05b7de16670f8a0cf0ac7    1
229ac14c6ee6d6a606264ebcc861beb7    1
                                   ..
31665bd5bc542687ab54c9fd9ed9917b    1
37fb9cce079a4887c96b59bae2011d05    1
1d815f3ad9b2017b6da2c4f1cbb1180b    1
6b05c8f3a25a71005d5eb9b51766c333    1
274fa6071e5e17fe303b9748641082c8    1
Name: count, Length: 99441, dtype: int64

Mas podem ter vários customer_unique_id para cada linha



customer_unique_id
8d50f5eadf50201ccdcedfb9e2ac8455    17
3e43e6105506432c953e165fb2acf44c     9
1b6c7548a2a1f9037c1fd3ddfed95f33     7
ca77025e7201e3b30c44b472ff346268     7
6469f99c1f9dfae7733b25662e7f1782     7
                                    ..
081f07439678af2da7755a0aa572154d     1
9c57789a6a587f4efcfb01e36c106014     1
9e4c21635f640562de1a6976374fba03     1
06c0ea6ee892364d1608ee47aa9f56a0     1
84732c5050c01db9b23e19ba39899398     1
Name: count, Length: 96096, dtype: int64

por exemplo:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
14186,1bd3585471932167ab72a84955ebefea,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
15321,a8fabc805e9a10a3c93ae5bff642b86b,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
16654,897b7f72042714efaa64ac306ba0cafc,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
36122,b2b13de0770e06de50080fea77c459e6,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
38073,42dbc1ad9d560637c9c4c1533746f86d,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
40141,dfb941d6f7b02f57a44c3b7c3fefb44b,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
48614,65f9db9dd07a4e79b625effa4c868fcb,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
52574,1c62b48fb34ee043310dcb233caabd2e,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
58707,a682769c4bc10fc6ef2101337a6c83c9,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP
67996,6289b75219d757a56c0cce8d9e427900,8d50f5eadf50201ccdcedfb9e2ac8455,4045,sao paulo,SP


In [222]:
print("as informações de um customer_unique_id mudam em diferentes linhas? ")
df_aux = (
    df_customers
    .groupby('customer_unique_id')
    .nunique()
    .drop(columns=['customer_id'])
    .sum(axis=1)
    .sort_values()
)
display(
    df_aux
)
print(f"Sim, {(df_aux>3).mean():2.2%} mudam. Por exemplo:")
display(
    df_customers.query("customer_unique_id=='d44ccec15f5f86d14d6a2cfa67da1975'")
)

as informações de um customer_unique_id mudam em diferentes linhas? 



customer_unique_id
0000366f3b9a7992bf8c76cfdf3221e2    3
aa53470c30cfd5c372dd45a57203e529    3
aa514cd67b53c21de47c7dc5972555e7    3
aa5014f5a39d8869707206c78f15ba07    3
aa4fc6bd5cf9d541b5a808cdada1301c    3
                                   ..
ee4a5dab94b427525ea24bc30d4442fb    6
8c21dd8c37144807c601f99f2a209dfb    6
2c45ab66a3dae52960147e76a35740ff    6
b9badb100ff8ecc16a403111209e3a06    7
d44ccec15f5f86d14d6a2cfa67da1975    9
Length: 96096, dtype: int64

Sim, 0.26% mudam. Por exemplo:


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
45851,844a7e5d2737f2f6667a5242486cbcb1,d44ccec15f5f86d14d6a2cfa67da1975,3533,sao paulo,SP
57951,105e31e3b36d904c70058578126c4d12,d44ccec15f5f86d14d6a2cfa67da1975,62800,aracati,CE
83347,488dd91f70d8404b2aa677cdc8a0de68,d44ccec15f5f86d14d6a2cfa67da1975,88371,navegantes,SC


In [ ]:
assert all(df_customers.customer_id != df_customers.customer_unique_id)

In [ ]:
# quantos pedidos por cada clientes:
n_customer_with_n_orders_gt_1 = (
    df_customers
    .merge(
        df_orders,
        on='customer_id',
        validate='1:m',
        how='left',
    )
    .groupby('customer_unique_id')
    .order_id.nunique()
    .sort_values()
     > 1
).sum()
n_customers = df_customers.customer_unique_id.nunique()
print(f'Dos {n_customers} apenas {n_customer_with_n_orders_gt_1}({n_customer_with_n_orders_gt_1/n_customers:2.2%}) tem mais de um pedido.')
print('Isso inviabiliza problemas a nível de clientes, pois há pouco histórico de compras do clintes. Por exemplo, churn de cliente')
print("Isto é, não existem clientes com histórico muito longo na plataforma")

Dos 96096 apenas 2997(3.12%) tem mais de um pedido.


## Preprocessamento: Trazendo info de vendedor e filtrando apenas pedidos

In [104]:
df_cust_pre = (
    df_customers
    .merge(
        df_orders[["order_id", "customer_id", "order_status","order_purchase_timestamp"]]
        .drop_duplicates(),
        on="customer_id",
        how='left',
        validate='1:1'
    )
    .query("order_status=='delivered'")
    .merge(
        df_items[['seller_id','order_id']]
        .drop_duplicates(),
        on=['order_id'],
        how='left',
        validate='1:m',
        indicator=True
    )
)
assert all(df_cust_pre._merge.astype("string").fillna("nulo") == "both")
df_cust_pre = df_cust_pre.drop(columns=['_merge', 'order_status','customer_id', 'order_id'])

In [105]:
df_cust_pre

,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_purchase_timestamp,seller_id
0,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,2017-05-16 15:05:35,7c67e1448b00f6e969d365cea6b010ab
1,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,2018-01-12 20:48:24,b8bc237ba3788b23da09c0f1f3a3288c
2,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,2018-05-19 16:07:45,7c67e1448b00f6e969d365cea6b010ab
3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,2018-03-13 16:06:38,7c67e1448b00f6e969d365cea6b010ab
4,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,2018-07-29 09:51:30,4a3ca9315b744ce9f8e9374361493884
...,...,...,...,...,...,...
97814,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP,2018-04-07 15:48:17,527801b552d0077ffd170872eb49683b
97815,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP,2018-04-04 08:20:22,3fd1e727ba94cfe122d165e176ce7967
97816,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE,2018-04-08 20:11:50,d9e7e7778b32987280a6f2cb9a39c57d
97817,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS,2017-11-03 21:08:33,4869f7a5dfa277a7dca6462dcf3b52b2


# Vendedores (base cadastral)

In [249]:
print(f"Existem {df_sel.seller_id.nunique()} vendedores na base.")

Existem 3095 vendedores na base.


In [250]:
df_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [260]:
n_sellers = df_sel.seller_id.nunique()
s = (
    df_sel
    .merge(df_items[['seller_id','order_id']], on=['seller_id'], how='left')
    .groupby('seller_id')
    .order_id.nunique()
)
n_sellers_with_gt_1 = len(s[s>1])
display(s.describe())
print(f"Dos {n_sellers} vendedores {n_sellers_with_gt_1}({n_sellers_with_gt_1/n_sellers:2.2%}) tem mais de uma venda")
print("Parece ser possível fazer um problema de machine learning a nível de vendedores")
print("Um exemplo é o risco do vendedor ficar sem vender nos próximos x meses")
print("Outro exemplo é o volume de vendas (quantidade ou monetário)"
      " de um vendedor nos próximos x meses")


count    3095.000000
mean       32.313409
std       105.139763
min         1.000000
25%         2.000000
50%         6.000000
75%        21.500000
max      1854.000000
Name: order_id, dtype: float64

Dos 3095 vendedores 2524(81.55%) tem mais de uma venda
Parece ser possível fazer um problema de machine learning a nível de vendedores
Um exemplo é o risco do vendedor ficar sem vender nos próximos x meses
Outro exemplo é o volume de vendas (quantidade ou monetário) de um vendedor nos próximos x meses


In [262]:
(
    df_sel
    .merge(df_items[['seller_id','order_id']],
           on=['seller_id'],
           how='outer',
           indicator=True)
    ._merge.value_counts(normalize=True)
)


_merge
both          1.0
left_only     0.0
right_only    0.0
Name: proportion, dtype: float64

## Preprocessamento: filtrando vendedores

In [141]:
(
    df_sel
    .merge(
        df_items[['seller_id','order_id']]
        .drop_duplicates(),
        on=['seller_id'],
        how='left',
        validate='m:m',
    )
    .merge(
        df_orders[["order_id", "order_status"]]
        .drop_duplicates(),
        on="order_id",
        how='left',
        validate='m:1'
    )
    .query("order_status=='delivered'")
    .drop_duplicates(['seller_id'])
    .drop(columns=["order_status","order_id"])
)

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
3,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
43,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
44,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
45,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
...,...,...,...,...
99983,98dddbc4601dd4443ca174359b237166,87111,sarandi,PR
99985,f8201cab383e484733266d1906e2fdfa,88137,palhoca,SC
99995,74871d19219c7d518d0090283e03c137,4650,sao paulo,SP
100001,e603cf3fec55f8697c9059638d6c8eb5,96080,pelotas,RS


# Pedidos

In [29]:
df_orders.columns


Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [245]:
df_orders.order_purchase_timestamp.min(),df_orders.order_purchase_timestamp.max()

('2016-09-04 21:15:19', '2018-10-17 17:30:18')

In [264]:
df_orders.order_id.value_counts()


order_id
e481f51cbdc54678b7cc49136f2d6af7    1
f01059d0d674e1282df4e8fbbe015aa2    1
fbc17f0f2a2125054d5ac5c22d2d5120    1
9373150545066777b1cd2bc20e93cf8e    1
917399e96f92268dfa2c0351b1b75fba    1
                                   ..
6b8986012d61963295ffa3ea869aff86    1
be879f757debd3b384b540daa6ddd97e    1
ac3ce7eda1246f39509a505242fcc169    1
e801a93b3904ca9e8350f176a037047b    1
66dea50a8b16d9b4dee7af250b4be1a5    1
Name: count, Length: 99441, dtype: int64

In [268]:
df_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14
...,...,...,...,...,...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41
112646,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53
112647,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95
112648,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72


In [270]:
df_items[['order_id','product_id','order_item_id']].value_counts()


order_id                          product_id                        order_item_id
00010242fe8c5a6d1ba2dd792cb16214  4244733e06e7ecb4970a6e2683c13e61  1                1
ab114f47094abcacaf5888844db683ff  1efd84ba20d23186f576a37baf64ae9d  1                1
ab14fdcfbe524636d65ee38360e22ce8  9571759451b1d780ee7c15012ea109d4  4                1
                                                                    3                1
                                                                    2                1
                                                                                    ..
553a1f1b74cf15fe6cb4a01a627ab41f  e8316a4667e5870c85e906b1f062bde1  1                1
553a0f2feba6fd288aaea2986ca2f94a  d9bdf643d95cb89844c0da1a0df1d16e  1                1
5539bd029cf95d97ba8f51f6f323c839  3bab56924ec762e9340478909312d5c7  1                1
553901a853048dcd33ec8de19f90c5d0  ac034106148bc5d510565d4b640f74c2  1                1
fffe41c64501cc87c801fd61db3f6244  350688d9dc1e7

## usar todos os pedidos ou apenas aqueles com determinado status na modelagem?

In [ ]:
# faz sentido considerar apenas os pedidos entregues
# já que os demais tem poucos casos
(
    df_orders
    .merge(df_items[['seller_id','order_id']].drop_duplicates(),
           on=['order_id'],
           how='left',
           indicator=True)
    .rename(columns={'_merge':'_merge_order_item'})
    [["_merge_order_item","order_status"]].value_counts(normalize=True)
    .sort_index()
)


_merge_order_item  order_status
left_only          canceled        0.001627
                   created         0.000050
                   invoiced        0.000020
                   shipped         0.000010
                   unavailable     0.005983
both               approved        0.000020
                   canceled        0.004574
                   delivered       0.970571
                   invoiced        0.003096
                   processing      0.002987
                   shipped         0.010994
                   unavailable     0.000069
Name: proportion, dtype: float64

In [288]:
df_orders.columns


Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

## Preprocessamento: Trazendo o id do vendedor para a base e info de cliente

In [89]:
df_orders_pre = (
    df_orders
    .query("order_status=='delivered'")
    .merge(
        df_items[['seller_id','order_id']]
        .drop_duplicates(),
        on=['order_id'],
        how='left',
        validate='1:m',
        indicator=True
    )
)
assert all(df_orders_pre._merge.astype("string").fillna("nulo") == "both")
df_orders_pre = df_orders_pre.drop(columns=['_merge',"customer_id"])

In [90]:
df_orders_pre

,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,seller_id
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,3504c0cb71d7fa48d967e0e4c94d59d9
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,289cdb325fb7e7f891c38608bf9e0962
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,4869f7a5dfa277a7dca6462dcf3b52b2
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,66922902710d126a0e7d26b0e3805106
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2c9e548be18521d1c43cde1c582c6de8
...,...,...,...,...,...,...,...,...
97814,9c5dedf39a927c1b2549525ed64a053c,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28 00:00:00,e24fc9fcd865784fb25705606fe3dfe7
97815,63943bddc261676b46f01ca7ac2f7bd8,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02 00:00:00,1f9ab4708f3056ede07124aad39a2554
97816,83c1379a015df1e13d02aae0204711ab,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27 00:00:00,d50d79cb34e38265a8649c383dcffd48
97817,11c177c8e97725db2631073c19f07b62,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15 00:00:00,a1043bafd471dff536d0c462352beb48


## target

In [10]:
s = catalog.load('pre_orders').cohort_info.value_counts().sort_index()[-12:]
max(s[s>s.quantile(0.05)].index)
last_cohort_target_available = (
    _cohort_offset(last_cohort_available,-performance_period)
)

[01/02/25 11:42:50] INFO     Loading data from pre_orders (ParquetDataset)...                   ]8;id=270210;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=336957;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

201808

In [15]:
catalog.load('pre_orders').cohort_info.value_counts().sort_index()

[01/02/25 13:59:40] INFO     Loading data from pre_orders (ParquetDataset)...                   ]8;id=922462;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=371567;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\


cohort_info
201609       3
201610     312
201612       1
201701     793
201702    1746
201703    2646
201704    2409
201705    3692
201706    3242
201707    4037
201708    4353
201709    4289
201710    4637
201711    7548
201712    5686
201801    7333
201802    6746
201803    7281
201804    7069
201805    6967
201806    6274
201807    6374
201808    6571
201809       1
Name: count, dtype: Int64

In [14]:
catalog.load("scoring.feature_table")

[01/02/25 13:59:16] INFO     Loading data from scoring.feature_table (ParquetDataset)...        ]8;id=190535;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=33102;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,seller_id,cohort,ord_total_orders_m3,ord_nunique_cohorts_m3,ord_mean_estimated_days_to_order_delivery_m3,ord_std_estimated_days_to_order_delivery_m3,ord_max_estimated_days_to_order_delivery_m3,ord_min_estimated_days_to_order_delivery_m3,ord_mean_days_to_order_approval_m3,ord_std_days_to_order_approval_m3,...,geo_min_distance_customer_seller_m3,geo_mean_distance_customer_seller_m6,geo_std_distance_customer_seller_m6,geo_max_distance_customer_seller_m6,geo_min_distance_customer_seller_m6,geo_mean_distance_customer_seller_m9,geo_std_distance_customer_seller_m9,geo_max_distance_customer_seller_m9,geo_min_distance_customer_seller_m9,sel_seller_state
0,3504c0cb71d7fa48d967e0e4c94d59d9,201801,12.0,2.0,16.416667,4.521833,22.0,11.0,0.166667,0.389249,...,18.566632,322.062512,526.838706,2708.402561,15.803204,322.062512,526.838706,2708.402561,15.803204,SP
1,66922902710d126a0e7d26b0e3805106,201801,38.0,3.0,22.210526,6.312398,50.0,12.0,0.210526,0.474079,...,4.101376,573.587674,457.430746,2551.745767,4.101376,585.590158,455.246476,2551.745767,4.101376,MG
2,8581055ce74af1daba164fdbd55a40de,201801,83.0,3.0,21.108434,7.065913,44.0,10.0,0.180723,0.521363,...,4.074288,468.701164,572.519755,2460.285152,4.074288,480.671681,559.926990,2460.285152,4.074288,SP
3,dc8798cbf453b7e0f98745e396cc5616,201801,8.0,2.0,19.750000,6.273527,30.0,10.0,0.375000,1.060660,...,56.487542,544.943056,658.722819,2215.081042,6.664046,437.449279,591.905927,2215.081042,6.502735,SP
4,16090f2ca825584b5a147ab24aa30c86,201801,28.0,3.0,20.571429,7.695254,34.0,7.0,0.321429,0.547964,...,36.448569,448.508300,593.425663,2179.033039,29.666845,461.449505,582.878122,2179.033039,29.666845,SP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17786,8dd386be0767c330276ea6a3f96532d3,201809,1.0,1.0,19.000000,NaN,19.0,19.0,0.000000,NaN,...,1393.442246,1393.442246,NaN,1393.442246,1393.442246,1393.442246,NaN,1393.442246,1393.442246,SC
17787,babcc0ab201e4c60188427cae51a5b8b,201809,1.0,1.0,18.000000,NaN,18.0,18.0,4.000000,NaN,...,218.635286,218.635286,NaN,218.635286,218.635286,218.635286,NaN,218.635286,218.635286,SP
17788,d263fa444c1504a75cbca5cc465f592a,201809,1.0,1.0,22.000000,NaN,22.0,22.0,0.000000,NaN,...,2304.357375,2304.357375,NaN,2304.357375,2304.357375,2304.357375,NaN,2304.357375,2304.357375,SP
17789,edf3fabebcc20f7463cc9c53da932ea8,201809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,819.907913,NaN,819.907913,819.907913,819.907913,NaN,819.907913,819.907913,SP


# Abt

In [4]:
catalog.load('abt').filter(regex='slope')

[12/29/24 07:32:21] INFO     Loading data from abt (ParquetDataset)...                          ]8;id=409451;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=719123;file:///home/bruno/Documents/Programming/Programming_projects/olist_project/.venv/lib/python3.11/site-packages/kedro/io/data_catalog.py#389\389]8;;\

,ord_slope_nunique_order_id_m3,ord_slope_nunique_order_id_m6,ord_slope_nunique_order_id_m9,itm_slope_sum_price_m3,itm_slope_count_product_id_m3,itm_slope_sum_price_m6,itm_slope_count_product_id_m6,itm_slope_sum_price_m9,itm_slope_count_product_id_m9
0,-3.5,-2.885714,-0.083333,-700.250,-4.0,-577.562857,-3.571429,-83.128333,-0.416667
1,3.0,0.371429,0.583333,127.150,3.5,37.697143,0.885714,28.768333,0.650000
2,-3.0,1.800000,0.900000,-331.755,-3.0,320.874571,1.800000,171.937833,0.900000
3,NaN,NaN,-0.216667,NaN,NaN,NaN,NaN,-6.806667,-0.316667
4,-30.0,3.285714,4.083333,-4843.595,-30.5,410.959714,3.314286,636.314500,4.016667
...,...,...,...,...,...,...,...,...,...
9194,0.0,0.085714,0.050000,0.000,0.0,12.857143,0.085714,7.500000,0.050000
9195,NaN,NaN,-0.050000,NaN,NaN,NaN,NaN,-1.470000,-0.050000
9196,NaN,-0.028571,0.016667,NaN,NaN,-1.711429,-0.028571,0.998333,0.016667
9197,0.0,0.085714,0.050000,0.000,0.0,5.991429,0.085714,3.495000,0.050000
